In [1]:
from qiskit_utils import *

## **Quantum Palindrome Checking**

In tis problem our aim is to devise a quantum algorithm which can recognise whether a given string sequence is palindromic or not. However here I will try to elaborate a technique using which we can not only check classical strings but also more genralised string of quantum alphabets. 

For an quantum algorithm that solves this problem for calssical binary strings check out the \href{https://github.com/qosf/monthly-challenges}{qosf-monthly-challenges} repo.

### **Quantum Alphabets**

In the usual classical sense an alphabet could be any finite set distinguishable symbols, for eg. $\mathcal{A}:$ {`a`, `b`, `c`} or $\mathcal{A}:$ {`1`, `2`, `3`}, etc. To generalise this idea to include quantum states we can say than an *quantum alphabet* is a set of distinct of quantum states, for eg. $\mathcal{A}:$ {$\ket{\phi_1}$,$\ket{\phi_2}$,$\ket{\phi_3}$}. However note that that here 'distinct' doesn't necessarily mean orthogonal, so even while the quantum states are distinct in the sense that $ \ket{\phi_2} \neq \ket{\phi_1}$, they might not be classicaly distiguishable.

For eg. $\mathcal{A}:$ {$ \ket{\phi_1} = \: \ket{0} + e^{-i \frac{\pi}{2}} \ket{1}$, $\ket{\phi_2} = \:\ket{0} + e^{-i \pi}\ket{1}$, $\ket{\phi_3} = \: \ket{0} + e^{-i \frac{3\pi}{2}}\ket{1}$}

### **Palindrome Checking**

Ususally the strings we need to check would be of the strings of the form `1324321` or `abdcdab` ,etc. The usual way of handling them is to iteratively compare the subsquent bits of the original string against the reversed string. Since classical strings whatever be number or alphabets can be always decompsed into binary representations, it suffices to run the palindrome checking algorithm on the binary representation of the string. For eg. given a binary string `101100` we can compare it bitwise with the string `001101` to check if it is palindromic.

However given a string of quantum alphabet , say `str :`$\mathtt{\ket{\phi_2}\ket{\phi_1}\ket{\phi_3}\ket{\phi_2}}$ a qubit-wise comparison might not be very effecient or even possible given the non-orthogoal nature of the quantum alphabets. Thus we will try to use a different approach more sutied for quantum states and also highlight whether such an algorithm could have a quantum advantage.

#### **Algorithm**
Before moving onto the quantum algorithm there are some assumptions we need to make,
1. The length of the string needs to be a dyadic i.e a power of 2. Though this might not very          convenient we can always pad a regular string to this form. 
2. 

Here I will describe the algorithm using an example and later point towards a possible generalisation.

**Step 1:** The first thing to do would be to load the elements of the quantum string into the memory using the QRAM implementations. For example if the string is `str :`$\mathtt{\ket{\phi_2}\ket{\phi_1}\ket{\phi_3}\ket{\phi_2}}$ the corresponding QRAM encoding would be, 
$$          \ket{\mathtt{data}} = \mathtt{ \ket{\phi_2}\ket{00} + \ket{\phi_1}\ket{01} + \ket{\phi_3}\ket{10} + \ket{\phi_2}\ket{11}  }
$$


**Step 2:**  Now to check whether the given string is a palidrome we should essentially compare the elements for which the data-registers are farthest apart, i.e compare the data at $\mathtt{\ket{00}}$ with the data at $\mathtt{\ket{11}}$ and the data at $\mathtt{\ket{01}}$ with the data at $\mathtt{\ket{10}}$ and so on. 

 Incase if all of them match up we can say that the quantum string is a palindrome, for our case the data-elements at $\mathtt{\ket{11}}$ and $\mathtt{\ket{00}}$ match up and is equal to $\mathtt{\ket{\phi_2}}$ wheras the data-elements at $\mathtt{\ket{10}}$ and $\mathtt{\ket{01}}$ are different.

 To do this comparison here we introduce an ancilla qubit $\ket{\mathtt{ancilla_1}}$ initiated in $\ket{\mathtt{0}}$, and then do CNOT on it controlled on the most significant bit of the adress register
 $$     \ket{\mathtt{data}}  =  \mathtt{ \ket{\phi_2}\ket{00}\ket{0} + \ket{\phi_1}\ket{01}\ket{0} + \ket{\phi_3}\ket{10}\ket{1} + \ket{\phi_2}\ket{11}\ket{1}}
 $$

Next controlled on $\mathtt{\ket{ancilla_1}}$ we do a CNOT on the elements of the adress register,
$$
  \ket{\mathtt{data}} = \mathtt{ \ket{\phi_2}\ket{00}\ket{0} + \ket{\phi_1}\ket{01}\ket{0} + \ket{\phi_3}\ket{01}\ket{1} + \ket{\phi_2}\ket{00}\ket{1}} \\
  
$$

this can reshuffled into 
$$ \ket{\mathtt{data}} =  \mathtt{ \ket{00}\left(\ket{\phi_2}\ket{0} + \ket{\phi_2}\ket{1} \right) + \ket{01} \left( \ket{\phi_1}\ket{0} + \ket{\phi_3}\ket{1} \right) }  \\
$$

finally do a $\mathbf{\hat{H}}$ operation on the $\mathtt{\ket{ancilla_1}}$, 

$$ \ket{\mathtt{data}} =  \mathtt{ \ket{00}\left( \left( \ket{\phi_2} + \ket{\phi_2} \right)\ket{0} +
\left( \ket{\phi_2} - \ket{\phi_2} \right)\ket{1} \right) + \ket{01}\left( \left( \ket{\phi_3} + \ket{\phi_1} \right)\ket{0} +
\left( \ket{\phi_3} - \ket{\phi_1} \right)\ket{1} \right)}
$$

**Step 3:** The last step makes the necessary comparison between the elements as required for this checking. We can check the result of the comparison directly by looking at the $\mathtt{\ket{ancilla_1}}$ register since for states matching up they can never be found in the $\mathtt{\ket{1}}$ state,

$$ \ket{\mathtt{data}} =  \mathtt{ \ket{00}\left( \left( \ket{\phi_2} + \ket{\phi_2} \right)\ket{0}
 \right) + \ket{01}\left( \left( \ket{\phi_3} + \ket{\phi_1} \right)\ket{0} +
\left( \ket{\phi_3} - \ket{\phi_1} \right)\ket{1} \right)}
$$

To make the comparison more complete we introduce another ancilla qubit $\mathtt{\ket{ancilla_2}}$ initiated in $\mathtt{\ket{1}}$, and operate on it a CNOT gate controlled on the  $\mathtt{\ket{ancilla_1}}$, thus obtaining

$$ \ket{\mathtt{data}} =  \mathtt{ \ket{00}\left( \left( \ket{\phi_2} + \ket{\phi_2} \right)\ket{0}\ket{1}
 \right) + \ket{01}\left( \left( \ket{\phi_3} + \ket{\phi_1} \right)\ket{0} +
\left( \ket{\phi_3} - \ket{\phi_1} \right)\ket{1} \right)}
$$

**Step 3:** 